In [9]:
import numpy as np
import torch
from torch import nn, optim
from dataset import get_twit_company_dataloaders, get_twit_sentiment_dataloaders, get_twit_company_sentiment_dataloaders
from model import LSTMTwitClassifier
import torch.nn.functional as F

# text, label = next(iter(dataloader_train))

In [10]:
device = "cuda"
use_wandb = True

lr = 0.001
embedding_size = 100
hidden_size = 100
epochs_cnt = 200
embeddings = "random"
lstm_layers = 1
dropout = 0.8
task = "text2sentiment" # "text2company" #
use_company_info = False
preprocessing = "tutorial"
use_stop_words = True
weight = [5, 3.5, 1, 1.5] # None

save_file = f"{epochs_cnt}epochs_{task}_{preprocessing}{f'_use_company' if use_company_info and task == 'text2sentiment' else ''}{'_'.join(map(str, weight)) + '_weight' if weight is not None else ''}x"

get_dataloaders = get_twit_company_dataloaders if task == "text2company" else \
    get_twit_sentiment_dataloaders if not use_company_info else get_twit_company_sentiment_dataloaders

dataset_train, dataloader_train, dataset_test, dataloader_test = get_dataloaders(embedding_dim=embedding_size,
                                                                                 embedding=embeddings,
                                                                                 preprocessing=preprocessing,
                                                                                 use_stop_words=use_stop_words)

model = LSTMTwitClassifier(4, embedding_dim=embedding_size, hidden_dim=hidden_size, dropout=dropout,
                           lstm_layers=lstm_layers,
                           additional_one_hot_arg=use_company_info and task == "text2sentiment")
model = model.to(device)

if use_wandb:
    import wandb

    wandb.init(project=task + '_twit_classification', entity='ars860')

    config = wandb.config
    config.loss = "BCE"
    config.optimizer = "Adam"
    config.learning_rate = lr
    config.hidden_size = hidden_size
    config.embedding_size = embedding_size
    config.embeddings = embeddings
    config.epochs = epochs_cnt
    config.dropout = dropout
    config.lstm_layers = lstm_layers
    config.stem = "snowballstemmer"
    config.preprocessing = preprocessing
    config.use_stop_words = use_stop_words
    if weight is not None:
        config.weight = weight

    if task == "text2sentiment":
        config.use_company_info = use_company_info

    wandb.watch(model)

criterion = nn.BCELoss(weight=torch.Tensor(weight).to(device) if weight is not None else None)
optimizer = optim.Adam(model.parameters(), lr=lr)

best_test_loss = 1000
best_test_accuracy = 0

def loss_on_test():
    global best_test_loss
    global best_test_accuracy

    correct = 0
    losses = np.zeros(len(dataloader_test))

    with torch.no_grad():
        model.eval()
        for i, (*args, target) in enumerate(dataloader_test):
            args = [arg.to(device) for arg in args]
            target = target.to(device)

            prediction = model(*args)
            prediction = F.softmax(prediction, dim=0)

            losses[i] = F.binary_cross_entropy(prediction, target.view(-1))
            if torch.argmax(prediction) == torch.argmax(target):
                correct += 1

            # predictions_cnt[torch.argmax(prediction)] += 1

            # if i % 100 == 0:
            #     print(f"Iter: {i}/{len(dataloader_test)}")

    model.train()
    test_loss = np.mean(losses)
    test_accuracy = correct / len(dataloader_test)

    if test_accuracy > best_test_accuracy:
        best_test_loss = test_loss
        best_test_accuracy = test_accuracy
        torch.save(model.state_dict(), f"learned_models/{save_file}_best_test.pt")

    if use_wandb:
        wandb.log({"test_loss": test_loss, "test_accuracy": test_accuracy})


losses = np.empty(100)
model.train()
for epoch in range(epochs_cnt):
    epoch_loss = np.zeros(len(dataloader_train))

    for i, (*args, target) in enumerate(dataloader_train):
        args = [arg.to(device) for arg in args]
        target = target.to(device)

        model.zero_grad()

        prediction = model(*args)
        prediction = F.softmax(prediction, dim=0)

        loss = criterion(prediction, target.view(-1))

        loss.backward()
        optimizer.step()

        loss = loss.detach().item()
        losses[i % 100] = loss
        epoch_loss[i] = loss

        if (i + 1) % 100 == 0:
            print(
                f"Epoch {epoch + 1}/{epochs_cnt}, iter: {i + 1}/{len(dataloader_train)}, mean loss: {np.mean(losses)}")
            if use_wandb:
                wandb.log({"loss": np.mean(losses)})

    if use_wandb:
        wandb.log({"epoch_loss": np.mean(epoch_loss)})
        loss_on_test()

# [model.get_word_embedding(word) for word in "hello_world".split(' ')]

Tweet ignored due to unreadability: Поиск от 
Tweet ignored due to unreadability: Новите 
Tweet ignored due to unreadability: 看見 
Tweet ignored due to unreadability: نظام جديد .. و جهاز جديد شكراً جزيلاً 
Tweet ignored due to unreadability: الجهاز الجديد عجيب   
Tweet ignored due to unreadability: يبدو ان طفرة الاجهزة الالكترونية القادمة ستكون بقيادة موتورولا ،، لاسيم بعد استحواذ قوقل عليها.   
Tweet ignored due to unreadability: Με συγχισες 
Tweet ignored due to unreadability: На сайте 
Tweet ignored due to unreadability: Настоящий твиттерянин как только попадает в толпу стремиться тут же как можно быстрее попасть в 
Tweet ignored due to unreadability: Доброе утро 
Tweet ignored due to unreadability: 【
Tweet ignored due to unreadability: رقم الفلو والفلورز والتويتات  للبيع لاعلى سعر 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ars86\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Tweet ignored due to unreadability: قال الرئيس التنفيذي لشركة 
Tweet ignored due to unreadability: Улучшим продукты компании 
Tweet ignored due to unreadability: نفسي يوم يعدي علي تويتر من غير مشاكل فنية 
Tweet ignored due to unreadability: ツイッター検索 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ars86\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\users\ars86\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/200, iter: 100/3360, mean loss: 1.3031138804135844
Epoch 1/200, iter: 200/3360, mean loss: 2.5816040808189427
Epoch 1/200, iter: 300/3360, mean loss: 1.014934973604977
Epoch 1/200, iter: 400/3360, mean loss: 0.1479249526152853
Epoch 1/200, iter: 500/3360, mean loss: 1.8702306636475259
Epoch 1/200, iter: 600/3360, mean loss: 0.15970223857089877
Epoch 1/200, iter: 700/3360, mean loss: 0.060828052923534416
Epoch 1/200, iter: 800/3360, mean loss: 0.03523302722262087
Epoch 1/200, iter: 900/3360, mean loss: 0.9973654297605754
Epoch 1/200, iter: 1000/3360, mean loss: 1.2377739647647832
Epoch 1/200, iter: 1100/3360, mean loss: 1.5525528037548064
Epoch 1/200, iter: 1200/3360, mean loss: 1.9799127134680747
Epoch 1/200, iter: 1300/3360, mean loss: 0.8509672784805298
Epoch 1/200, iter: 1400/3360, mean loss: 0.44435552509501575
Epoch 1/200, iter: 1500/3360, mean loss: 1.0887078601308167
Epoch 1/200, iter: 1600/3360, mean loss: 0.5911839162930846
Epoch 1/200, iter: 1700/3360, mean loss: 0.20

In [11]:
model.eval()

print("Testing on train")

correct = 0
predictions_cnt = [0, 0, 0, 0]

with torch.no_grad():
    for i, (*args, target) in enumerate(dataloader_train):
        args = [arg.to(device) for arg in args]
        target = target.to(device)

        prediction = model(*args)
        prediction = F.softmax(prediction, dim=0)

        if torch.argmax(prediction) == torch.argmax(target):
            correct += 1

        predictions_cnt[torch.argmax(prediction)] += 1

        if i % 100 == 0:
            print(f"Iter: {i}/{len(dataloader_train)}")

print(f"Accuracy {correct / len(dataloader_train)}")

torch.save(model.state_dict(), f"learned_models/{save_file}_final.pt")

if use_wandb:
    wandb.run.summary.train_accuracy = correct / len(dataloader_train)
    wandb.run.summary.classified_as = {
        "apple": predictions_cnt[0],
        "google": predictions_cnt[1],
        "microsoft": predictions_cnt[2],
        "twitter": predictions_cnt[3]
    }

    wandb.run.summary.best_test_accuracy = best_test_accuracy
    wandb.run.summary.best_test_loss = best_test_loss

    artifact = wandb.Artifact('final_model', type='model')
    artifact.add_file(f'learned_models/{save_file}_final.pt')
    wandb.run.log_artifact(artifact)
    artifact = wandb.Artifact('best_test_model', type='model')
    artifact.add_file(f'learned_models/{save_file}_best_test.pt')
    wandb.run.log_artifact(artifact)

    wandb.finish()

Testing on train
Iter: 0/3360
Iter: 100/3360
Iter: 200/3360
Iter: 300/3360
Iter: 400/3360
Iter: 500/3360
Iter: 600/3360
Iter: 700/3360
Iter: 800/3360
Iter: 900/3360
Iter: 1000/3360
Iter: 1100/3360
Iter: 1200/3360
Iter: 1300/3360
Iter: 1400/3360
Iter: 1500/3360
Iter: 1600/3360
Iter: 1700/3360
Iter: 1800/3360
Iter: 1900/3360
Iter: 2000/3360
Iter: 2100/3360
Iter: 2200/3360
Iter: 2300/3360
Iter: 2400/3360
Iter: 2500/3360
Iter: 2600/3360
Iter: 2700/3360
Iter: 2800/3360
Iter: 2900/3360
Iter: 3000/3360
Iter: 3100/3360
Iter: 3200/3360
Iter: 3300/3360
Accuracy 0.9669642857142857


loss,0.10054
_runtime,4042
_timestamp,1636728198
_step,6999
epoch_loss,0.21639
test_loss,0.95775
test_accuracy,0.69277
train_accuracy,0.96696
best_test_accuracy,0.73193
best_test_loss,0.63433


loss,▅▄▃▂█▂▂▂▂▃▃▂▃▂▁▄▂▁▂▁▅▂▂▂▁▁▁▂▂▂▁▂▂▂▃▁▂▂▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss,██▅▄▃▃▂▃▂▂▂▂▂▁▂▁▁▁▁▁▁▂▁▁▂▁▁▂▁▂▁▁▁▁▁▁▁▂▁▂
test_loss,█▂▃▂▃▃▃▂▂▂▂▁▃▁▂▃▁▃▂▂▃▁▃▃▃▃▂▄▄▅▅▄▄▆▄▃▄▄▅▄
test_accuracy,▁▄▅▆▆▇▆▇▇▇▇█▇█▇▇█▇▇█▇█▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇


In [12]:
print("Testing on test")

correct = 0
predictions_cnt = [0, 0, 0, 0]

with torch.no_grad():
    for i, (*args, target) in enumerate(dataloader_test):
        args = [arg.to(device) for arg in args]
        target = target.to(device)

        prediction = model(*args)
        prediction = F.softmax(prediction, dim=0)

        if torch.argmax(prediction) == torch.argmax(target):
            correct += 1

        predictions_cnt[torch.argmax(prediction)] += 1

        if i % 100 == 0:
            print(f"Iter: {i}/{len(dataloader_test)}")

print(f"Accuracy {correct / len(dataloader_test)}")


Testing on test
Iter: 0/332
Iter: 100/332
Iter: 200/332
Iter: 300/332
Accuracy 0.6927710843373494
